# **Implémentation du modèle**
---

## Introduction

Ce notebook contient les codes nécessaire à la mise en place du modèle. Le modèle est inspiré de celui de Iacoviello et Navarro : 

- La première étape consiste à identifier les chocs de politique monétaire, en prenant le réside de la régression du taux d'intérêt sur l'écart d'inflation par rapport à la cible et l'output gap ou le chômage. En réalité Iacoviello et Navarro utilisent des lags et valeurs présentes de l'inflation, des spreads de crédit, du PIB et des lags des taux des fonds fédéraux.
Il nous manque donc l'output gap, l'inflation et le chômage de la zone euro
- La deuxième étape consiste à estimer l'impact de ces chocs sur l'activité économique (PIB, emploi ...) en régressant ces variables sur les chocs et des variables de contrôles (4 lags du PIB et des trends linéaires et quadratiques)



## Identification des chocs

In [9]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

Problème: dans Iacoviello et Navarro, ils régressent le taux d'intérêt sur le PIB etc des US seulement. Or on est dans la zone euro ici, doit-on faire une régression de panel ? 

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

data = pd.read_pickle('data_final.pkl')

# Sélection des colonnes utiles
colonnes_utiles = ['federal_funds_rate', 'inflation', 'log_euroarea_gdp', 'corporate_spreads', 'log_foreign_gdp', 'lagged_federal_funds', 'time']
data = data[colonnes_utiles]

# Création d'une variable de tendance quadratique
data['time_squared'] = data['time'] ** 2

# Définition de rt=federal funds rate
Y = data['federal_funds_rate']

# Définir les variables indépendantes
X = data[['inflation', 'log_euroarea_gdp', 'corporate_spreads', 'log_foreign_gdp', 'lagged_federal_funds', 'time', 'time_squared']]
X = sm.add_constant(X)  # Ajout d'une constante pour l'intercept

# Regression
model = sm.OLS(Y, X).fit()

# Extraire les résidus comme chocs monétaires
data['monetary_shocks'] = model.resid

# Afficher le résumé de la régression
print(model.summary())

# Sauvegarder les résultats dans un fichier CSV
data[['time', 'monetary_shocks']].to_csv('identified_monetary_shocks.csv', index=False)
